# Joint vs Individual Linear Regression Loadings

First, some notation. Be warned that I am going to take advantage of the typographical similarity between certain Greek and Roman letters (classicists and pedants, avert thy eyes!). Other than that, however, my notation should be very natural to anyone who's taken a college-level linear models course.

Afterward, I'll run some experiments where I prove (or at least state) some theoretical (ground-truth) results about the "Greek" quantities, then demonstrate them using their "Roman" counterparts calculated on toy datasets. To give a trivial example, to demonstrate that $\sigma(\gamma) \geq 0$, I could show that $s(y) = 0.42$ (which is nonnegative) for some specific toy dataset.


## The Greeks: Ground-Truth Data-Generating Process

Let $\gamma,\, \chi_1,\, \chi_2,\, \varepsilon \mid \beta_0,\, \beta_1,\, \beta_2$ be a finite-variance Multivariate Normal "data-generating process" (basically, a vector of real-valued random variables) such that $\gamma = \beta_0 + \beta_1\chi_1 + \beta_2\chi_2 + \varepsilon$ where $\varepsilon$ is i.i.d. white noise.

Let $\sigma(\cdot)$ represent standard deviation, $\sigma^2(\cdot)$ represent variance, $\sigma^2(\cdot,\, \cdot)$ represent covariance, and $\rho(\cdot,\, \cdot)$ represent correlation. Let $\Sigma$ be the variance-covariance matrix between $\chi_1$ and $\chi_2$ i.e.
$$\begin{pmatrix} \sigma^2(\chi_1) & \sigma^2(\chi_1,\, \chi_2) \\ \sigma^2(\chi_1,\, \chi_2) & \sigma^2(\chi_2)\end{pmatrix},$$ and $\Omega$ be the corresponding correlation matrix.

Importantly, assume that $|\rho(\chi_1,\, \chi_2)| \neq 1$.


## The Romans: Practical Calculations On Observed Data

Suppose we observe $N$ different "draws" or "samples" from this process. Arrange the draws of $\gamma$ into a column vector of real numbers $y := [y_n]$, the draws of $\chi_1$ into a column vector of real numbers $x_1 := [x_{n,1}]$, the draws of $\chi_2$ into a column vector of real numbers $x_2 := [x_{n,2}]$, and the draws of $\varepsilon$ into a column vector of real numbers $e := [e_n]$, over $n \in [1,\, N]$. Further, arrange the $x$'s into an $N \times 3$ matrix of real numbers $X := [1,\, x_1,\, x_2]$ whose first column is all ones (AKA "constant" AKA "intercept").

Let the vector of real numbers $b := [b_0,\, b_1,\, b_2] := (X^\top X)^{-1} X^\top y$ be the coefficients from an OLS regression of $y$ onto $X$ [1]. More generally, define $\texttt{ols}$ such that e.g. $\texttt{ols}(y,\, [1,\, x_1,\, x_2]) := [b_0, b_1, b_2] =: b$.

Finally, let $s$, $s^2$, $r$, $S$, and $U$ be the usual Bessel-corrected "sample" estimators of their "population" counterparts in Greek above.


Footnotes
---------
[1] Take this formula for granted. Recall that if $\varepsilon$ is Normally distributed, then OLS yields the MLE for $\beta$ given $y,\, X$. On the other hand if $\varepsilon$ is Laplace-distributed, then instead LAD would yield the MLE.

In [2]:
from typing import Tuple, Union
import numpy as np
import pandas as pd
import statsmodels.api as sm


def gen_data(mean: Tuple[float]=(0, 0), std: Tuple[float]=(1, 1), corr: float=0, n: int=10_000_000, seed: int=42):
    """
    Generate a toy dataset where y = x1 + x2 + white_noise.

    input
    -----
    mean: tuple[float] (default 0), ground-truth means of the x's.
    std: tuple[float] (default 1), ground-truth standard deviations of the x's.
    corr: float (default 0), ground-truth correlation between the x's.
    n: int (default 10 million), number of data points to generate.
    seed: int (default 42), random seed.

    output
    ------
    pd.DataFrame(columns=["y", "x1", "x2", "x1+x2", "white_noise"]).
    """
    mean = pd.Series({"x1": mean[0], "x2": mean[1], "white_noise": 0})

    std = pd.Series({"x1": std[0], "x2": std[1], "white_noise": 1})
    # diagonal matrix with std's on the diagonal
    std_ = pd.DataFrame(np.diag(std), index=std.index, columns=std.index)

    corr = pd.DataFrame({
        "x1": {"x1": 1, "x2": corr, "white_noise": 0},
        "x2": {"x1": corr, "x2": 1, "white_noise": 0},
        "white_noise": {"x1": 0, "x2": 0, "white_noise": 1}
    })

    cov = std_ @ corr @ std_

    np.random.seed(seed=seed)
    df = pd.DataFrame(np.random.multivariate_normal(mean=mean, cov=cov, size=n), columns=mean.index)
    df.loc[:, "x1+x2"] = df["x1"] + df["x2"]
    df.loc[:, "y"] = df["x1+x2"] + df["white_noise"]
    return df[["y", "x1", "x2", "x1+x2", "white_noise"]]


def ols(y: pd.Series, x: Union[pd.Series, pd.DataFrame], decimals: int=2):
    """Get OLS coefficient vector."""
    return np.round(sm.OLS(endog=y, exog=sm.add_constant(x), hasconst=True).fit().params, decimals=decimals)


# example
df = gen_data()
ols(y=df["y"], x=df["x1"])

const   -0.0
x1       1.0
dtype: float64

## Result 0.0

Suppose $\rho(\chi_1,\, \chi_2) = 0$. Then, $\beta_i = \sigma^{-2}(\chi_i)\sigma^2(\chi_i,\, \gamma)$. This is the familiar formula for a univariate regression slope, otherwise stated as $\beta_i = \frac{\texttt{Cov}(\chi_i,\, \gamma)}{\texttt{Var}(\chi_i)}$. Notice how similar this looks to the $(X^\top X)^{-1} X^\top y$ formula.

Pf: Trivial. For example, consider the data-generating process $\gamma,\, \chi_1,\, \varepsilon^\prime$ where $\gamma = \beta_0 + \beta_2\texttt{E}(\chi_2) + \beta_1\chi_1 + \varepsilon + \beta_2(\chi_2 - \texttt{E}(\chi_2))$ which we write as $\beta_0^\prime + \beta_1\chi_1 + \varepsilon^\prime$ with $\sigma^2(\varepsilon^\prime) = \sigma^2(\varepsilon) + \beta_2^2\sigma^2(\chi_2)$. This latter form is amenable to univarate OLS, which by construction must be consistent with our original multivariate formulation.

### Corollary 0.0.C
Let $[a_0,\, a_1] := \texttt{ols}(y,\, [1, x_1])$, $[b_0,\, b_2] := \texttt{ols}(y,\, [1, x_2])$, $[c_0,\, c_1,\, c_2] := \texttt{ols}(y,\, [1, x_1,\, x_2])$. We will have $c_0 = a_0 + b_0$, $c_1 = a_1$, and $c_2 = b_2$ iff $r(x_1,\, x_2) = 0$.

## Result 0.0.1

Suppose further that $\texttt{E}(\chi_1) = 0 = \texttt{E}(\chi_2)$. Then, $[\beta_1,\, \beta_2] = \Sigma^{-1}\Sigma_\gamma$ where $\Sigma_\gamma$ the is the column vector $[\sigma^2(\chi_1,\, \gamma) ,\, \sigma^2(\chi_2,\, \gamma)]$ of covariances between the $\chi$'s and $\gamma$.

Pf: Exercise of doing the OLS matrix calculations "pictorally" for the bivariate case with $N$ observations left to the reader. Thence, prove the theoretical result by representing the underlying Multivariate Normal distribution as an $\infty \times 4$ matrix. Fair disclosure: I have myself done the first step but haven't done the second step which requires a level of linear algebra that I just barely got to in college. However, from my vague recollection of that semester, I can't imagine how it could fail to work.

## Result 0.1

Suppose $\rho(\chi_1,\, \chi_2) \neq 0$. Then, the conclusion of Result 0.0 will not hold.

Pf: Take for granted that the univariate OLS slope estimate for e.g. $\beta_1$ has omitted-variable bias of $\beta_2 \sigma^{-2}(\chi_1) \sigma^2(\chi_1,\, \chi_2)$. We set $\beta_2 = 1$ and assume finite variance, and in this scenario are supposing that $\rho(\chi_1,\, \chi_2) \neq 0 \implies \sigma^2(\chi_1,\, \chi_2) \neq 0$. Therefore, $a_1$ is a biased estimator of $\beta_1$. However, $a_1 := s^{-2}(x_1)s^2(x_1,\, y)$ is an *un*biased estimator of $\sigma^{-2}(\chi_1)\sigma^2(\chi_1,\, \gamma)$. Hence, $\beta_1 \neq \sigma^{-2}(\chi_1)\sigma^2(\chi_1,\, \gamma)$, QED.

## Result 1

Let $[a_0,\, a_1] := \texttt{ols}(y,\, [1, x_1])$, $[b_0,\, b_2] := \texttt{ols}(y,\, [1, x_2])$, $[c_0,\, c_{1+2}] := \texttt{ols}(y,\, [1, x_1+x_2])$. Under what conditions will we have $c_0 = a_0 + b_0$ and $c_{1+2} = a_1 + b_2$?

Let's start backward:
$$a_1 = \frac{s^2(x_1,\, y)}{s^2(x_1)}$$
$$b_2 = \frac{s^2(x_2,\, y)}{s^2(x_2)}$$
$$a_1 + b_2 = \frac{s^2(x_1,\, y)}{s^2(x_1)} + \frac{s^2(x_2,\, y)}{s^2(x_2)} = \frac{s^2(x_2)s^2(x_1,\, y) + s^2(x_1)s^2(x_2,\, y)}{s^2(x_1)s^2(x_2)}$$
$$c_{1+2} = \frac{s^2(x_1 + x_2,\, y)}{s^2(x_1 + x_2)} = \frac{s^2(x_1,\, y) + s^2(x_2,\, y)}{s^2(x_1) + s^2(x_2) + 2s^2(x_1,\, x_2)}$$
Hence we want
$$\frac{s^2(x_2)s^2(x_1,\, y) + s^2(x_1)s^2(x_2,\, y)}{s^2(x_1)s^2(x_2)} = \frac{s^2(x_1,\, y) + s^2(x_2,\, y)}{s^2(x_1) + s^2(x_2) + 2s^2(x_1,\, x_2)}$$
Clearly this is underidentified so let's assert that $s(x_1) = s = s(x_2)$, whereby we get
$$\frac{s^2(x_1,\, y) + s^2(x_2,\, y)}{s^2} = \frac{s^2(x_1,\, y) + s^2(x_2,\, y)}{2s^2 + 2s^2(x_1,\, x_2)}$$
Now the numerators match, so we just need to make the denominators match
$$s^2 = 2s^2 + 2s^2(x_1,\, x_2) = 2s^2 + 2r(x_1,\, x_2)s^2 = 2(1 + r(x_1,\, x_2))s^2$$
$$1 = 2(1 + r(x_1,\, x_2))$$
$$-0.5 = r(x_1,\, x_2)$$
So if their standard deviations are the same, we need $x_1$ and $x_2$ to be $-0.5$ correlated if we want to slopes to add up.